In [ ]:
! cp -r ../test2 ./
! cp -r ../train_ref ./
! ls -al

In [ ]:
import cv2
import numpy as np
from skimage.morphology import skeletonize
from sklearn.metrics import precision_score, recall_score, accuracy_score, confusion_matrix
from scipy.spatial import KDTree
import concurrent.futures
import os
import gc

In [ ]:
import cv2
import numpy as np
from skimage.morphology import skeletonize
from sklearn.metrics import precision_score, recall_score, accuracy_score, confusion_matrix
from scipy.spatial import KDTree
import os
import time

# 1. 이미지 전처리
def preprocess_image(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    blurred = cv2.GaussianBlur(image, (5, 5), 0)
    _, binary = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    skeleton = skeletonize(binary // 255).astype(np.uint8)
    return skeleton

# 2-1. Minutiae (end, bif) 검출 (벡터화 사용)
def detect_minutiae(skeleton):
    start_time = time.time()
    
    minutiae = {'end': [], 'bif': []}

    kernel = np.array([[1, 1, 1], [1, 10, 1], [1, 1, 1]])
    filtered = cv2.filter2D(skeleton, -1, kernel)

    minutiae['end'] = np.argwhere(filtered == 11).tolist()
    minutiae['bif'] = np.argwhere(filtered == 13).tolist()
    
    end_time = time.time()
    print(f"detect_minutiae processing time: {end_time - start_time:.4f} seconds")

    return minutiae

# 3-1. 매칭 (Euclidean Distance 계산)
def match_fingerprints(test_minutiae, train_minutiae, threshold):
    test_points = np.array(test_minutiae['end'] + test_minutiae['bif'])
    train_points = np.array(train_minutiae['end'] + train_minutiae['bif'])

    if len(test_points) == 0 or len(train_points) == 0:
        return float('inf')  # 매칭할 포인트가 없으면 무한대 반환

    # Calculate Euclidean distances
    kd_tree = KDTree(train_points)
    distances, _ = kd_tree.query(test_points)
    average_distance = np.mean(distances)

    return average_distance

# 4. Metric 계산
def calculate_metrics(true_labels, predicted_labels):
    precision = precision_score(true_labels, predicted_labels)
    recall = recall_score(true_labels, predicted_labels)
    accuracy = accuracy_score(true_labels, predicted_labels)
    cm = confusion_matrix(true_labels, predicted_labels)
    FAR = cm[0, 1] / (cm[0, 1] + cm[0, 0]) if (cm[0, 1] + cm[0, 0]) > 0 else 0
    FRR = cm[1, 0] / (cm[1, 1] + cm[1, 0]) if (cm[1, 1] + cm[1, 0]) > 0 else 0
    return precision, recall, accuracy, FAR, FRR

# 데이터 폴더 경로
train_folder = 'train_ref/'
test_folder = 'test2/'

def process_image_pair(test_image_path, train_image_paths, threshold):
    try:
        test_skeleton = preprocess_image(test_image_path)
        test_minutiae = detect_minutiae(test_skeleton)
        
        best_match_score = float('inf')
        best_train_image = None
        
        for train_image_path in train_image_paths:
            train_skeleton = preprocess_image(train_image_path)
            train_minutiae = detect_minutiae(train_skeleton)
            
            match_score = match_fingerprints(test_minutiae, train_minutiae, threshold)
            
            if match_score < best_match_score:
                best_match_score = match_score
                best_train_image = train_image_path
        
        return best_match_score
    except Exception as e:
        print(f'Error processing {test_image_path}: {e}')
        return float('inf')

def main():
    train_images = [os.path.join(train_folder, f) for f in os.listdir(train_folder) if f.endswith('.BMP')]
    test_images = [os.path.join(test_folder, f) for f in os.listdir(test_folder) if f.endswith('.BMP')]

    threshold = 10  # 임계값 설정

    all_true_labels = []
    all_predicted_labels = []

    start_time = time.time()
    
    # 순차적으로 처리
    for test_image_path in test_images:
        print(f'Processing {test_image_path}')
        best_match_score = process_image_pair(test_image_path, train_images, threshold)
        all_true_labels.append(1)  # 예시로 모든 매치를 1로 설정
        all_predicted_labels.append(1 if best_match_score < threshold else 0)

    elapsed_time = time.time() - start_time
    print(f'Total processing time: {elapsed_time:.2f} seconds')

    precision, recall, accuracy, FAR, FRR = calculate_metrics(all_true_labels, all_predicted_labels)

    print(f'Precision: {precision}, Recall: {recall}, Accuracy: {accuracy}, FAR: {FAR}, FRR: {FRR}')

if __name__ == "__main__":
    main()
